In [1]:
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.engine.topology import Layer
import keras.initializers 
from keras.layers import Lambda, Input 
from keras.models import Model

C:\Users\Ni-Co\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [20]:
class Kepler(Layer):

    def __init__(self, equinoctial_orbit, **kwargs):
        self.orbit = equinoctial_orbit
        self.mu = K.constant(3.986004418e14)
        super(Kepler, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='equinoctial_orbit', 
                                      shape=(1, 6),
                                      initializer=keras.initializers.constant(self.orbit),
                                      trainable=True)
        super(Kepler, self).build(input_shape)  # Be sure to call this at the end

    def call(self, time):
        semi_major_axis = self.orbit[0]
        mean_motion = K.sqrt(self.mu/semi_major_axis)/semi_major_axis
        orbit_update = [0, 0, 0, 0, 0, mean_motion] * time
        return self.orbit + orbit_update

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 6)

In [28]:
time = Input(shape=(1,), dtype=np.float32, name='date')
ephemeris = Kepler([7999e3, 1e-4, 0, 0.2, 0.1, 0.7], name='propagation')(time)
propagator = Model(time, ephemeris)
propagator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
date (InputLayer)            (None, 1)                 0         
_________________________________________________________________
propagation (Kepler)         (None, 6)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [31]:
equinoctial_elements = propagator.predict(np.arange(0, 7200, 600))

In [5]:
def normalize_angle(angle):
    two_pi = 2*np.pi
    return angle - two_pi*tf.floor((angle + np.pi)/two_pi)

In [36]:
def compute_dF(F, m):
    sinF, cosF = tf.sin(F), tf.cos(F)
    return (F + h*cosF - k*sinF - m)/(1 - h*sinF - k*cosF)

sess = tf.InteractiveSession()
h, k, F_test = 0.1, 0.2, np.pi/4

m_test = F_test + h*tf.cos(F_test) - k*tf.sin(F_test)

newton_iteration = lambda F: (F-compute_dF(F, m_test))

F, dF = m_test, compute_dF(m_test, m_test)

for i in range(10):
    F = newton_iteration(F) 
    print(F.eval())
print("ref: {}, computed: {}".format(F_test, F.eval()))
sess.close()


C:\Users\Ni-Co\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0.7855916
0.7853982
0.7853982
0.7853982
0.7853982
0.7853982
0.7853982
0.7853982
0.7853982
0.7853982
ref: 0.7853981633974483, computed: 0.7853981852531433


In [13]:
def solve_kepler_equation(h, k, m):
    def compute_dF(F):
        sinF, cosF = tf.sin(F), tf.cos(F)
        return (F + h*cosF - k*sinF - m)/(1 - h*sinF - k*cosF)

    has_converged = lambda F: tf.reduce_any(tf.greater(compute_dF(F), 1e-12))
    newton_iteration = lambda F: F-compute_dF(F)

    F0 = m
    F = tf.while_loop(has_converged, newton_iteration, [F0], maximum_iterations=20, name='newton_kepler')
    return F

In [48]:
class UtilsTest(tf.test.TestCase):
    def test_solve_kepler_equation(self):
        with self.test_session():    
            h_grid, k_grid, m_grid = np.meshgrid(
                np.arange(0, 0.5, 0.01), np.arange(0, 0.2, 0.01), np.arange(-np.pi/2, np.pi/2, 0.01))
            h, k, m = tf.constant(h_grid.flatten()), tf.constant(k_grid.flatten()), tf.constant(m_grid.flatten())
            F = solve_kepler_equation(h, k, m)
            
            recomputed_m = F + h*tf.cos(F) - k*tf.sin(F)
            self.assertArrayNear(m.eval(), recomputed_m.eval(), 1e-10)

UtilsTest().test_solve_kepler_equation()

In [ ]:
tests.run

In [2]:
sess = tf.InteractiveSession()

In [45]:
a, h, k, p, q, m = tf.unstack(equinoctial_elements, num=6, axis=-1)

p2 = p*p
q2 = q*q
_2pq = 2*p*q

normalization_coef = 1/(1 + p2 + q2)

print(normalization_coef.eval())
f = normalization_coef*tf.stack([1 - p2 + q2, _2pq, -2*p], axis=1)

g = normalization_coef*tf.stack([_2pq, 1 + p2 - q2, 2*q])



F = solve_kepler_equation(h, k, m)
sinF, cosF = tf.sin(F), tf.cos(F)

mu = K.constant(3.986004418e14)
n = K.sqrt(mu/a)/a
h2 = h*h
k2 = k*k
hk = h*k
sqrt1mh2mk2 = K.sqrt(1 - h2 - k2)
b = 1/(1+sqrt1mh2mk2)

denL = 1 - h*sinF - k*cosF
sinL = ((1 - k2*b)*sinF + hk*b*cosF - h)/denL
cosL = ((1 - h2*b)*cosF + hk*b*sinF - k)/denL

r = a*(1 - h*sinF - k*cosF)

x = r*cosL
y = r*sinL
xDot = -n*a*(h + sinL)/sqrt1mh2mk2
yDot = n*a*(k + cosL)/sqrt1mh2mk2
    



[0.952381 0.952381 0.952381 0.952381 0.952381 0.952381 0.952381 0.952381
 0.952381 0.952381 0.952381 0.952381]


ValueError: Dimensions must be equal, but are 12 and 3 for 'mul_232' (op: 'Mul') with input shapes: [12], [12,3].

In [17]:


a, h, k, p, q, m = tf.unstack(equinoctial_elements, num=6)
f, g = self.__compute_equinoctial_frame(p, q)
F = solve_kepler_equation(h, k, m)
sinF, cosF = tf.sin(F), tf.cos(F)


n = K.sqrt(mu/a)/a
h2 = h*h
k2 = k*k
hk = h*k
sqrt1mh2mk2 = K.sqrt(1 - h2 - k2)
b = 1/(1+sqrt1mh2mk2)

denL = 1 - h*sinF - k*cosF
sinL = ((1 - k2*b)*sinF + hk*b*cosF - h)/denL
cosL = ((1 - h2*b)*cosF + hk*b*sinF - k)/denL

r = a*(1 - h*sinF - k*cosF)

x = r*cosL
y = r*sinL
xDot = -n*a*(h + sinL)/sqrt1mh2mk2
yDot = n*a*(k + cosL)/sqrt1mh2mk2

position = x*f + y*g
velocity = xDot*f + yDot*g

def __compute_equinoctial_frame(self, p, q):
    p2 = p*p
    q2 = q*q
    _2pq = 2*p*q

    normalization_coef = 1/(1 + p2 + q2)

    f = normalization_coef*tf.stack([1 - p2 + q2, _2pq, -2*p])
    g = normalization_coef*tf.stack([_2pq, 1 + p2 - q2, 2*q])
    return f, g

NameError: name 'self' is not defined

In [37]:
class Cartesian(Layer):

    def __init__(self, **kwargs):
        self.mu = K.constant(3.986004418e14)
        super(Cartesian, self).__init__(**kwargs)

    def build(self, input_shape):
        super(Cartesian, self).build(input_shape)

    def call(self, equinoctial_elements):
        
        a, h, k, p, q, m = tf.unstack(equinoctial_elements, num=6, axis=-1)
        f, g = self.__compute_equinoctial_frame(p, q)
        F = solve_kepler_equation(h, k, m)
        sinF, cosF = tf.sin(F), tf.cos(F)
        
        n = K.sqrt(self.mu/a)/a
        h2 = h*h
        k2 = k*k
        hk = h*k
        sqrt1mh2mk2 = K.sqrt(1 - h2 - k2)
        b = 1/(1+sqrt1mh2mk2)
        
        denL = 1 - h*sinF - k*cosF
        sinL = ((1 - k2*b)*sinF + hk*b*cosF - h)/denL
        cosL = ((1 - h2*b)*cosF + hk*b*sinF - k)/denL
        
        r = a*(1 - h*sinF - k*cosF)
        
        x = r*cosL
        y = r*sinL
        xDot = -n*a*(h + sinL)/sqrt1mh2mk2
        yDot = n*a*(k + cosL)/sqrt1mh2mk2
        
        position = x*f + y*g
        velocity = xDot*f + yDot*g
        return tf.stack([position, velocity])
    
    def __compute_equinoctial_frame(self, p, q):
        p2 = p*p
        q2 = q*q
        _2pq = 2*p*q

        normalization_coef = 1/(1 + p2 + q2)
                
        f = normalization_coef*tf.stack([1 - p2 + q2, _2pq, -2*p])
        g = normalization_coef*tf.stack([_2pq, 1 + p2 - q2, 2*q])
        return f, g
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], 6)

In [38]:
time = Input(shape=(1,), dtype=np.float32, name='date')
equinoctial_coordinates = Kepler([7999e3, 1e-4, 0, 0.2, 0.1, 0.7], name='propagation')(time)
cartesian_coordinates = Cartesian(name='cartesian')(equinoctial_coordinates)
propagator = Model(time, cartesian_coordinates)
propagator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
date (InputLayer)            (None, 1)                 0         
_________________________________________________________________
propagation (Kepler)         (None, 6)                 6         
_________________________________________________________________
cartesian (Cartesian)        (None, 6)                 0         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [39]:
propagator.predict(np.arange(0, 7200, 600))

ValueError: could not broadcast input array from shape (2,3,12) into shape (12,3,12)